<a href="https://colab.research.google.com/github/rashida048/ApacheSpark-BigDataAnalytics/blob/main/lr_svc_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 62 kB/s 
     |████████████████████████████████| 198 kB 53.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=922d4b85fccda5877aad77c6154cf7f5887d803d0a1ddd9900fe55935d1fd669
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [13]:
task3_total_strt = timeit.default_timer()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [14]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.getOrCreate()

In [15]:
import re
import numpy as np

In [16]:
def buildArray(listOfIndices):
  returnVal = np.zeros(20000)

  for index in listOfIndices:
    returnVal[index] = returnVal[index] + 1

  mysum = np.sum(returnVal)

  returnVal = np.divide(returnVal, mysum)
  return returnVal

In [17]:
def changeBinary(x):
  if "AU" in x:
    return 1
  else:
    return 0

In [18]:
def conversion(rd):
  validLines = rd.filter(lambda x: 'id' in x and 'url=' in x)
  keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
  regex = re.compile('[^a-zA-Z]')
  keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))
  allWords = keyAndListOfWords.flatMap(lambda x: [(y, 1) for y in x[1]])
  allCounts = allWords.reduceByKey(lambda x, y: x+y)
  topWords = allCounts.top(20000, lambda x: x[1])
  topWordsK = sc.parallelize(range(20000))
  dictionary = topWordsK.map (lambda x : (topWords[x][0], x))
  allWordsWithDocID = keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))
  allDictionaryWords = allWordsWithDocID.join(dictionary)
  justDocAndPos = allDictionaryWords.map(lambda x: x[1])
  allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()
  allDocsAsNumpyArrays = allDictionaryWordsInEachDoc.map(lambda x: (x[0], buildArray(list(x[1]))))
  xy = allDocsAsNumpyArrays.map(lambda x: (changeBinary(x[0]), x[1]))
  xy1 = xy.map(lambda x: (x[0], x[1].tolist()))
  xy_df = xy1.toDF(["lab", "feat"])
  return (xy, xy_df)

In [19]:
import timeit
strt_total = timeit.default_timer()

strt_read = timeit.default_timer()
rd = sc.textFile("SmallTrainingData.txt")
stp_read = timeit.default_timer()
print((stp_read - strt_read)/60, "minutes to read the training data")

0.0014213288333327985 minutes to read the training data


In [20]:
conv = conversion(rd)

In [21]:
df = conv[1]

In [22]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

In [23]:
convers = udf(lambda x: Vectors.dense(x), VectorUDT())
df1 = df.withColumn("features", convers("feat")).drop("feat")

In [24]:
from pyspark.ml.feature import ChiSqSelector
selector=ChiSqSelector(numTopFeatures =10000, percentile=0.9, featuresCol="features", outputCol='selectedFeatures', labelCol= 'lab')

In [25]:
model = selector.fit(df1)

In [26]:
res = model.transform(df1)

In [27]:
select_df = res.select('lab', 'selectedFeatures')

In [29]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LinearSVC

In [30]:
lr_train_strt = timeit.default_timer()
lr_red = LogisticRegression(featuresCol="selectedFeatures", labelCol='lab',
                        maxIter = 20)
lrmodel_new = lr_red.fit(select_df)

lr_train_stp = timeit.default_timer()
print((lr_train_stp-lr_train_strt)/60, "minutes to train the logistic regression with reduced features")

3.79750204705 minutes to train the logistic regression with reduced features


In [32]:
strt_read_test = timeit.default_timer()
rd1 = sc.textFile("SmallTrainingData.txt")
stp_read_test = timeit.default_timer()
print((stp_read_test - strt_read_test)/60, "minutes to read the testing data")

0.0010036462166662355 minutes to read the testing data


In [33]:
conv1 = conversion(rd1)

In [34]:
test_df = conv1[1]

In [35]:
convers1 = udf(lambda x: Vectors.dense(x), VectorUDT())
test_df1 = test_df.withColumn("features", convers("feat")).drop("feat")

In [36]:
model1 = selector.fit(test_df1)

In [37]:
select_test_df = model.transform(test_df1).select('lab', 'selectedFeatures')

In [39]:
lr_test_strt = timeit.default_timer()
lr_selected_pred = lrmodel_new.transform(select_test_df)

multi_eval = MulticlassClassificationEvaluator(labelCol="lab", metricName = "accuracy")
print('Logistic Regression Accuracy on Selected Features:', multi_eval.evaluate(lr_selected_pred))

multi_eval1 = MulticlassClassificationEvaluator(labelCol="lab", metricName = "f1")
print('Logistic Regression F1 on Selected Features:', multi_eval1.evaluate(lr_selected_pred))

lr_test_stp = timeit.default_timer()

lr_test_reduced = (lr_test_stp - lr_test_strt)/60
print(lr_test_reduced, "minutes to test the logistic regression with reduced features")

Logistic Regression Accuracy on Selected Features: 1.0
Logistic Regression F1 on Selected Features: 1.0
3.5288748549833335 minutes to test the logistic regression with reduced features


In [40]:
task3_total_stp = timeit.default_timer()
total_logistic = (task3_total_stp - task3_total_strt)/60
print(total_logistic, "minutes total to complete the logistic regression")

25.350520008233335 minutes total to complete the logistic regression


In [41]:
svc_train_stp = timeit.default_timer()

lsvc = LinearSVC(featuresCol="selectedFeatures", labelCol='lab', maxIter=10, regParam=0.1)
lsvcModel = lsvc.fit(select_df)

svc_train_strt = timeit.default_timer()

svc_train = (svc_train_strt - svc_train_stp)/60
print(svc_train, "minutes to train the support vector with reduced features")

3.683523200600003 minutes to train the support vector with reduced features


In [42]:
svc_test_stp = timeit.default_timer()

svc_select_pred = lsvcModel.transform(select_df)

multi_eval1 = MulticlassClassificationEvaluator(labelCol="lab", metricName = "accuracy")
print('SVC Accuracy on selected features:', multi_eval1.evaluate(svc_select_pred))

multi_eval_f1 = MulticlassClassificationEvaluator(labelCol="lab", metricName = "f1")
print('SVC F1 on selected features:', multi_eval1.evaluate(svc_select_pred))

svc_test_strt = timeit.default_timer()
svc_test = (svc_test_strt - svc_test_stp)/60
print(svc_test, "minutes to test the support vector with reduced features")

SVC Accuracy on selected features: 1.0
SVC F1 on selected features: 1.0
3.548390619966661 minutes to test the support vector with reduced features


In [44]:
end_time = timeit.default_timer()
total_time = (end_time - task3_total_strt)/60

print(total_time - (lr_train_stp-lr_train_strt)/60 - lr_test_reduced, "minutes total to complete SVC with reduced feature")

27.883456125916666
